In [1]:
!pip install pyspark==3.0.1 py4j==0.10.9

     |████████████████████████████████| 204.2 MB 43 kB/s 
     |████████████████████████████████| 198 kB 66.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612246 sha256=d411dab90e818e5a1fd4a683ae1e33e1a2d9eb6f6449c2ecfc3c0ed3e0eb5c5b
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.ml.stat import Correlation
from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('spotify_classification')\
        .getOrCreate()

In [4]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = spark.read.json("drive/My Drive/spot_data.json")

In [6]:
from pyspark.ml.feature import Bucketizer
bucketBorders=[0,10,20,30,40,50,60,70,80,90,100]

In [7]:
bucketer=Bucketizer().setSplits(bucketBorders).setInputCol("popularity_track").setOutputCol("label")
df_buck = bucketer.transform(df)

In [15]:
from pyspark.ml.feature import OneHotEncoder, VectorAssembler, StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf, StringType
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LinearSVC, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [18]:
assembler = VectorAssembler( 
inputCols=['age',
 'duration_ms',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'time_signature',
 'sum_artist_followers',
 'sum_artist_popularity'], 
outputCol="features")
output=assembler.transform(df_buck)

In [20]:
final_data = output.select( "features", "label")

In [21]:
train, test = final_data.randomSplit([0.7, 0.3], seed = 10)

In [22]:
lsvc = LinearSVC(maxIter=10, regParam=0.1)

In [23]:
ovr = OneVsRest(classifier=lsvc)

In [24]:
ovrModel = ovr.fit(train)

In [25]:
predictions = ovrModel.transform(test)

In [26]:
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")

In [27]:
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.697343


In [28]:
accuracy

0.30265708746721404